In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
# from torch.utils.tensorboard import SummaryWriter

In [4]:
df = pd.read_csv('/home/kaleb/Qualcom_comp/data/hackathon_train.csv', encoding='cp949', index_col=0)

df

,User_ID,Gender,Age,MBTI,Q_number,Answer
Data_ID,,,,,,
1,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
2,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
3,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
4,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
5,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...,...,...
14384,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...
14385,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...
14386,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...


In [5]:
# converting age to one-hot encoded columns
df = pd.get_dummies(df, columns=['Age'], prefix='cat')
df

,User_ID,Gender,MBTI,Q_number,Answer,cat_20,cat_30,cat_40
Data_ID,,,,,,,,
1,1,1,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...,0,1,0
2,1,1,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...,0,1,0
3,1,1,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...,0,1,0
4,1,1,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...,0,1,0
5,1,1,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...,0,1,0
...,...,...,...,...,...,...,...,...
14384,240,0,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...,0,0,1
14385,240,0,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...,0,0,1
14386,240,0,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...,0,0,1


In [6]:
# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:40])
    test_df_list.append(df[df['User_ID']==idx][40:])
    
train_df = pd.concat(train_df_list, ignore_index=True)
test_df = pd.concat(test_df_list, ignore_index=True)

In [7]:
train_df

,User_ID,Gender,MBTI,Q_number,Answer,cat_20,cat_30,cat_40
0,1,1,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...,0,1,0
1,1,1,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...,0,1,0
2,1,1,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...,0,1,0
3,1,1,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...,0,1,0
4,1,1,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...,0,1,0
...,...,...,...,...,...,...,...,...
9595,240,0,ISTJ,36,<아니다> 저는 즐거운 파티나 행사로 일주일 피로를 푸는 편이 아닙니다. 이유는 그...,0,0,1
9596,240,0,ISTJ,37,<중립> 저는 미술관 가는 일을 좋아하지 않습니다. 이유는 미술 작품에 관심이 많이...,0,0,1
9597,240,0,ISTJ,38,<그렇다> 저는 다른 사람의 감정을 이해하기 힘들 때가 많습니다. 이유는 각자의 살...,0,0,1
9598,240,0,ISTJ,39,<아니다> 저는 매일 할 일을 계획하지 않습니다. 이유는 매일매일보다는 크게 한 건...,0,0,1


In [8]:
# model_name = 'kykim/bert-kor-base'
# model_name = 'monologg/kobigbird-bert-base'
# model_name = 'beomi/kcbert-base'
model_name = 'snunlp/KR-BERT-char16424' # MAIN EMBEDDING BERT MODEL
# model_name = 'kykim/albert-kor-base'
# model_name = 'klue/bert-base'
# model_name = 'klue/roberta-base'
# model_name = 'Twitter/twhin-bert-base'
# model_name = 'skt/kobert-base-v1'
# model_name = 'skt/kogpt2-base-v2'
# model_name = 'EleutherAI/polyglot-ko-1.3b'

In [9]:

train_result = torch.load(f'/home/kaleb/Qualcom_comp/{model_name.replace("/", "_")}_train_embed_regular.pt')[0]
test_result = torch.load(f'/home/kaleb/Qualcom_comp/{model_name.replace("/", "_")}_test_embed_regular.pt')[0]

In [10]:
def set_random(SEED=0):
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    np.random.seed(SEED)
    random.seed(SEED)

class MyDataset(Dataset):
    def __init__(self, data, label, label_idx=0):
        self.data = data
        self.label = label
        self.label_idx = label_idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], torch.tensor(self.label[idx][self.label_idx])
    
def convert_mbti_to_label(mbti: str):
    """
    :param mbti: string. length=4
    :return:
    """
    stand = 'ISTJ'  # [0, 0, 0, 0]
    result = []
    for i in range(4):
        if stand[i] == mbti[i]:
            result.append(0)
        else:
            result.append(1)

    return result

# def convert_label_to_mbti(num, label_idx):
#     stand = 'ISTJ'
#     mbti = stand[label_idx]

In [11]:
# from sklearn import decomposition
# # apply pca to 768 to make it 128
# pca = decomposition.PCA(n_components=256)
# train_result = pca.fit_transform(train_result[0].detach().cpu().numpy())
# test_result = pca.fit_transform(test_result[0].detach().cpu().numpy())

In [12]:
# added_cols = ['Gender'] + [col for col in train_df.columns if col.startswith('cat')]# or col.startswith('pred_is')]

added_cols = []
nums = 48
for i in range(nums):
    added_cols += ['Gender']
    if i % 3 == 0:
        added_cols += [col for col in train_df.columns if col.startswith('cat')]# or col.startswith('pred_is')]

col_data = train_df[added_cols].values
test_col_data = test_df[added_cols].values

# convert the numerical columns to tensors
col_data = torch.tensor(col_data).float().cuda(0)
test_col_data = torch.tensor(test_col_data).float().cuda(1)


In [13]:
print(added_cols)

['Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender', 'Gender', 'cat_20', 'cat_30', 'cat_40', 'Gender', 'Gender']


In [14]:
# from sklearn.svm import SVC
from torch.optim import Adam
# from thundersvm import SVC as ThunderSVC
import cudf
# from cuml.svm import SVC
from cuml import *
import joblib
from sklearn.metrics import roc_auc_score
gpu_id = 0
cudf.set_allocator("managed")
cudf.cuda.select_device(gpu_id)
def main(label_idx=0, epochs=50, name='IE', mlp=None, svm_model=None, test_number=None, chkpt_path=None, svm_path=None):
    print(col_data.shape[1])

    if chkpt_path is not None:
        mlp.load_state_dict(torch.load(chkpt_path))
    
    mlp = mlp.cuda(0)
    # Define the optimizer
    optimizer = Adam(mlp.parameters(), lr=1e-2)

    # input to the MLP -> col_data (they can be on cuda:0) -> output -> 768
    # then concatenate col_data and train_result[0] to give train_result_added
    # and feed the concatenated tensor to the svm
    train_label = train_df['MBTI'].map(convert_mbti_to_label)
    # convert a pandas series whose data values are list -> to a 2D numpy array
    train_label_2d = np.array(train_label.tolist())
    train_label_1d = train_label_2d[:, label_idx]
    test_label = test_df['MBTI'].map(convert_mbti_to_label)
    test_label_2d = np.array(test_label.tolist())
    test_label_1d = test_label_2d[:, label_idx]
    # Training
    
    # test_col_data = test_col_data.cuda(0)
    # col_data_merged = torch.cat([col_data, test_col_data.cuda(0)], dim=0)
    # labels_merged = np.concatenate([train_label_1d, test_label_1d], axis=0)
    # # test_result = test_result.cuda(0)
    # result_merged = torch.cat([train_result, test_result.cuda(0)], dim=0)
    # test_col_data = test_col_data.cuda(0)
    
    # svm_model = SVC(kernel='rbf', C=1, gamma=0.085)
    # hinge_losses = []
    # min_hinge_loss = np.inf
    max_auc = 0

    for epoch in range(epochs):
        
        # if mlp is in cuda:1 move it to cuda:0
        
        if mlp[0].weight.device != torch.device('cuda:0'):
            mlp = mlp.cuda(0)
        # Forward pass through the MLP
        mlp_output_train = mlp(col_data) # output has shape of (9600, 768)
        # print(f'passing {col_data} to the mlp')

        # Concatenate the output of the MLP with the additional features
        concatenated_features_train = np.hstack([mlp_output_train.detach().cpu().numpy(), train_result.detach().cpu().numpy()]) # shape of (9600, 768 + 768)
        # print(f'concatenating mlp output with the embedded features')

        # Train the SVM
        svm_model.fit(concatenated_features_train, train_label_1d) # shape of (9600, 768 + 768)

        # Compute the hinge loss
        hinge_loss = torch.nn.functional.multi_margin_loss(mlp_output_train, torch.from_numpy(train_label_1d).cuda(0), margin=1.0)
        

        # Backward pass and optimization
        optimizer.zero_grad()
        hinge_loss.backward()
        optimizer.step()


        # Testing
        with torch.no_grad():
            # first move the mlps to the cuda:1

            # mlp = mlp.cuda(1)
            mlp_output_test = mlp(test_col_data.cuda(0))
            concatenated_features_test = np.hstack([mlp_output_test.detach().cpu().numpy(), test_result.detach().cpu().numpy()])
            accuracy = svm_model.score(concatenated_features_test, test_label_1d)
            test_prob_pred = svm_model.predict_proba(concatenated_features_test)
            # decision_values = svm_model.decision_function(concatenated_features_test)
            roc_auc = roc_auc_score(test_label_1d, test_prob_pred[:, 1])
            
            
            if roc_auc > max_auc:
                max_auc = roc_auc
                # torch.save(mlp.state_dict(), f'./{name}_model_{test_number}.pth')
                # # also save it using joblib
                # joblib.dump(svm_model, f'./{name}_svm_{test_number}.pkl')
            
            # print(f'saving the models with the minimum hinge loss: {min_hinge_loss}')
            
        if (epoch+1) % 5 == 0:    
            print(f'Epoch [{epoch+1}/{epochs}], Hinge Loss: {hinge_loss.item()}, Accuracy: {accuracy}, ROC AUC: {roc_auc}')
    return max_auc, accuracy

/home/kaleb/miniconda3/lib/python3.10/site-packages/cudf/utils/utils.py:218: FutureWarning: The cudf.set_allocator function is deprecated and will be removed in a future release. Please use rmm.reinitialize (https://docs.rapids.ai/api/rmm/stable/api.html#rmm.reinitialize) instead. Note that `cudf.set_allocator(allocator="managed")` is equivalent to `rmm.reinitialize(managed_memory=True)`.
  warnings.warn(


In [15]:
# Train all
# suppress the warnings
import warnings
warnings.filterwarnings("ignore")
test_number = 204
import csv
MBTI = ['IE', 'SN', 'TF', 'JP']
mlp = nn.Sequential(nn.Linear(col_data.shape[1], 786//2),
                          nn.ReLU(),        
                          nn.Linear(786//2, 786),
                          )

set_random(42)
best_accuracies = {}
result_dir = f'./result_{test_number}_{model_name.replace("/", "_")}'
i = MBTI.index('IE') # change this to 'SN', 'TF', 'JP'
svm_model_ie = SVC(kernel='rbf', C=1.5, gamma=0.05, probability=True)
svm_model_sn = SVC(kernel='rbf', C=2.5, gamma=0.025, probability=True)
svm_model_tf = SVC(kernel='rbf', C=1.0, gamma=0.085, probability=True)
svm_model_jp = SVC(kernel='rbf', C=3.0, gamma=0.05, probability=True)

result, _ = main(label_idx=i, epochs=200, name=MBTI[i], mlp=mlp, svm_model=svm_model_ie, test_number=test_number)
best_accuracies[MBTI[i]] = result
print(f'Best accuracy for {MBTI[i]}: {result}')

# print(best_accuracies)

96
Epoch [5/200], Hinge Loss: 0.00105286727193743, Accuracy: 0.5, ROC AUC: 0.63873046875
Epoch [10/200], Hinge Loss: 0.0010300134308636189, Accuracy: 0.5041666626930237, ROC AUC: 0.6354475911458333
Epoch [15/200], Hinge Loss: 0.0010194211499765515, Accuracy: 0.5541666746139526, ROC AUC: 0.673154296875
Epoch [20/200], Hinge Loss: 0.001032668282277882, Accuracy: 0.6265624761581421, ROC AUC: 0.6784342447916666
Epoch [25/200], Hinge Loss: 0.0010006155353039503, Accuracy: 0.6343749761581421, ROC AUC: 0.7416221788194444
Epoch [30/200], Hinge Loss: 0.0009980399627238512, Accuracy: 0.65625, ROC AUC: 0.7585682508680557
Epoch [35/200], Hinge Loss: 0.0009768693707883358, Accuracy: 0.692187488079071, ROC AUC: 0.7664507378472222
Epoch [40/200], Hinge Loss: 0.0009788223542273045, Accuracy: 0.6916666626930237, ROC AUC: 0.7673621961805556
Epoch [45/200], Hinge Loss: 0.0009839421836659312, Accuracy: 0.692187488079071, ROC AUC: 0.7674034288194446
Epoch [50/200], Hinge Loss: 0.0009836069075390697, Accura

In [16]:
print(f'Best accuracy for {MBTI[i]}: {result}')

# after changing the hyperparameters to the ones stated on our paper, the results are:
# Best accuracy for SN: 0.7206195746527777
# Best accuracy for TF: 0.7281000434027778
# Best accuracy for JP: 0.7227734375

Best accuracy for IE: 0.7674034288194446
